<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900">**노트북 5: 멀티모달(Multimodal) 시퀀스 모델링**</font>

### **모달리티(Modality)란?**

멀티모달 아키텍처를 본격적으로 살펴보기 전에, 먼저 "모달리티"가 무엇인지 이해할 필요가 있습니다.  
**모달리티(Modality)**는 데이터의 표현 방식과 구조에 따라 분류되는 **정보의 유형**입니다.

| 모달리티 종류 | 설명 |
|---------------|------|
| **텍스트** | 단어나 토큰의 순차적 시퀀스 |
| **이미지** | 픽셀로 구성된 2차원 배열 |
| **오디오** | 시간축 기반의 연속적인 음파 데이터 |
| **비디오** | 시간에 따라 변화하는 이미지 프레임의 시퀀스 |
| **센서 데이터** | IoT 기기나 환경 센서에서 수집되는 수치 정보 |

### **우리가 주목할 모달리티**

이 노트북에서는 주로 아래의 두 가지 모달리티를 다룹니다:

- **텍스트**  
  → 자연어 문장은 순차적 의미 구조를 갖고 있으며, 토큰 시퀀스로 표현됩니다.

- **이미지**  
  → 픽셀 값들의 2차원 배열로 표현되며, 지역적 특징(디테일)과 전역적 의미(전체 구성)를 동시에 담고 있습니다.

### **멀티모달 모델링이란?**

멀티모달 모델링은 **서로 다른 모달리티를 동시에 처리하거나 상호보완적으로 활용**하는 방식입니다.

예를 들어,  
- 사용자가 **이미지**를 입력하고 모델이 이에 대해 **자연어 설명**을 생성하는 **이미지 캡셔닝(Image Captioning)**  
- **텍스트**로 질문하고, 그에 해당하는 **이미지**를 생성하는 **텍스트-투-이미지(Text-to-Image)**  
- 또는 둘 모두를 입력으로 사용해 **멀티모달 질의응답(VQA)**을 수행하는 작업 등

이처럼 서로 다른 입력 모달리티 간의 정보 흐름을 잘 통합하고 해석하는 것이 핵심입니다.

### **멀티모달 모델의 목표**

멀티모달 시스템의 최종 목표는 다음과 같습니다:

> **각 모달리티에서 전달되는 정보를 일관되고 효과적인 표현으로 변환하고, 이들을 결합하여 복잡한 과제를 해결하는 것**


### **실생활 예시**

- 이미지에 대한 설명 생성 (Image Captioning)  
- 텍스트 설명을 바탕으로 이미지 생성 (Text-to-Image Generation)  
- 상품 리뷰의 텍스트와 이미지를 함께 고려한 감정 분석  
- 동영상에서 특정 행동 탐지  
- 로봇에게 "왼쪽에 있는 빨간 컵을 집어"와 같은 명령을 이해시키기 위한 비전-언어 통합

## **Part 5.2: BLIP 기반 이미지 캡셔닝 실습**

이번 실습에서는 HuggingFace에서 제공하는 **BLIP 모델**을 사용해,  
이미지를 입력하면 자연어로 설명하는 **이미지 캡셔닝(Image Captioning)** 작업을 수행해보겠습니다.


In [ ]:
# 이미지 캡셔닝 모델과 프로세서 로드
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
import torch

# Hugging Face에서 BLIP 모델과 프로세서를 로드합니다
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 예시 이미지 불러오기
url = "https://raw.githubusercontent.com/salesforce/BLIP/main/demo/demo.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert('RGB')

# 이미지 출력
image.show()

###  캡션 생성하기

# 이미지 → 모델 입력 전처리
inputs = processor(image, return_tensors="pt").to(model.device)

# 모델을 통해 설명 생성
out = model.generate(**inputs)

# 결과 출력
print("📝 생성된 설명:", processor.decode(out[0], skip_special_tokens=True))

### 요약

| 구성 요소 | 설명 |
|-----------|------|
| **BLIP** | 이미지와 텍스트를 동시에 학습해 자연어 생성 및 질의응답에 활용 가능 |
| **Processor** | 이미지를 토큰화하고 모델 입력 형식으로 변환 |
| **generate()** | 이미지로부터 텍스트 생성 수행 |

BLIP은 사전 학습을 통해 시각적 개념을 언어로 표현하는 능력을 학습했습니다.  
이처럼 멀티모달 모델은 인간처럼 "보는 것"과 "설명하는 것"을 연결할 수 있습니다

# <font color="#76b900">**마무리**</font> 

이번 파트에서는 멀티모달 시퀀스 모델링의 핵심 개념과 적용 사례,  
특히 이미지 → 텍스트 변환을 다루는 이미지 캡셔닝(Image Captioning) 문제를 통해 실습해보았습니다.  

BLIP와 같은 멀티모달 모델은 단순히 이미지를 분류하거나 인식하는 것을 넘어,  
의미 기반의 자연어 생성으로 확장된다는 점에서 매우 강력한 표현력을 가지고 있습니다.

In [ ]:
## 작업이 완료되면 실행해주세요!

# IPython 모듈을 불러옵니다.
# IPython은 대화형 파이썬 실행 환경을 제공하는 고급 파이썬 쉘입니다.
import IPython

# 현재 실행 중인 IPython 인스턴스(커널)를 가져옵니다.
app = IPython.Application.instance()

# 현재 실행 중인 IPython 커널을 안전하게 종료합니다.
# True 매개변수는 재시작 없이 완전히 종료하라는 의미입니다.
app.kernel.do_shutdown(True)

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>